In [ ]:
import os
import cv2
import torch
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration

device = "cuda:0"

processor = Blip2Processor.from_pretrained(
    os.path.join(os.environ["SCRATCH"], "mq_libs/blip2")
)
model = Blip2ForConditionalGeneration.from_pretrained(
    os.path.join(os.environ["SCRATCH"], "mq_libs/blip2"),
    torch_dtype=torch.float16,
)
model.to(device)


/srv/beegfs02/scratch/aarslan_data/data/mambaforge/envs/mq_analysis/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-26 10:14:40.726256: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-26 10:14:40.727758: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-26 10:14:40.936224: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-26 10:14:41.673494: I tensorflow/core/platform/cpu_featu

In [ ]:
clip_id = "0076e425-bdb6-48b3-b4d3-695089ac9800"
question = "What is the person in this picture doing?"
frame_id = 10458

cap = cv2.VideoCapture(os.path.join(os.environ["SCRATCH"], "ego4d_data/v2/clips", clip_id + ".mp4"))
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id - 1)
_, frame = cap.read()

frame = Image.fromarray(frame[:, :, ::-1])

with torch.no_grad():
    all_generated_texts = []
    preprocessed_frames_batch_dict = processor(
        images=[
            frame
        ],
        text=["Question: " + question + " Answer:"],
        return_tensors="pt",
    ).to(device, torch.float16)
    generated_ids = model.generate(**preprocessed_frames_batch_dict)
    generated_text = processor.batch_decode(
        generated_ids, skip_special_tokens=True
    )[0].strip()

display(frame)
print(generated_text)

